In [9]:
# dependencies

import pandas as pd
import numpy as np
import json
import os

In [10]:
# search the dir for JSON files, and ask the user as to which of the files to open for analysis

file_list = []
x = 0

for file in os.listdir("HeroesOfPymoli"):
    if file.endswith(".json"):
        file_list.insert(x, os.path.join(file))
        x += x

file_list.sort()

for file_name in file_list:
    print("[" + str(file_list.index(file_name)) + "] " + file_name)

selection = int(input("Which file do you want to open? "))
jsonpath = ('HeroesOfPymoli/' + file_list[selection])

[0] purchase_data.json
[1] purchase_data2.json
Which file do you want to open? 0


In [11]:
# read JSON file

json_df = pd.read_json(jsonpath)
json_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [12]:
# Player Count - number of players based on unique'SN'

player_count = len(json_df["SN"].value_counts())
print("Total Number of Players: ", player_count)

Total Number of Players:  573


In [13]:
# Purchasing Analysis (Total)

unique_item_count = len(json_df["Item ID"].value_counts())

total_number_purchases = len(json_df["SN"].value_counts())
total_revenue = json_df["Price"].sum()
average_purchase_price = total_revenue / total_number_purchases

print("Number of Unique Items: ", unique_item_count)
print("Average Purchase Price: ", np.round(average_purchase_price, decimals=2))
print("Total Number of Purchases: ", total_number_purchases)
print("Total Revenue: ", np.round(total_revenue, decimals=2))

Number of Unique Items:  183
Average Purchase Price:  3.99
Total Number of Purchases:  573
Total Revenue:  2286.33


In [29]:
# Gender Demographics

gender_df = json_df.drop_duplicates(['SN'], keep='first')
gender_counts_df = pd.DataFrame(gender_df["Gender"].value_counts())
gender_counts_df = gender_counts_df.rename(columns={"Gender": "Number of Players"})
gender_counts_df['Percent'] = gender_counts_df["Number of Players"] / player_count * 100
print(gender_counts_df)
# Need to format percentage

                       Number of Players    Percent
Male                                 465  81.151832
Female                               100  17.452007
Other / Non-Disclosed                  8   1.396161


In [20]:
# Purchasing Analysis (Gender)

gender_analysis_df = gender_counts_df.rename(columns={"Number of Players": "Purchase Count"})
gender_analysis_df['Total Purchase Value'] = (json_df.groupby('Gender'))['Price'].sum()
gender_analysis_df['Average Purchase Price'] = gender_analysis_df['Total Purchase Value'] / gender_analysis_df['Purchase Count']
gender_analysis_df['Normalized Total Purchase Value'] = gender_analysis_df['Total Purchase Value'] / (json_df.groupby('Gender'))['SN'].count()
print(gender_analysis_df)

                       Purchase Count    Percent  Total Purchase Value  \
Male                              465  81.151832               1867.68   
Female                            100  17.452007                382.91   
Other / Non-Disclosed               8   1.396161                 35.74   

                       Average Purchase Price  Normalized Total Purchase Value  
Male                                 4.016516                         2.950521  
Female                               3.829100                         2.815515  
Other / Non-Disclosed                4.467500                         3.249091  


In [23]:
# Age Demographics

bins = [0, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 119]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '>64']
age_df = json_df
age_df['Age Group'] = pd.cut(age_df["Age"], bins, labels=group_names)
grouped_age_groups = age_df.groupby('Age Group')
purchase_per_group_df = pd.DataFrame(grouped_age_groups['SN'].count()) # unique SN?
renamed_purchase_per_group_df = purchase_per_group_df.rename(columns = {"Age Group": "Age Group", "SN": "Purchase Count"})
renamed_purchase_per_group_df['Total Purchase Value'] = (age_df.groupby('Age Group'))['Price'].sum()
renamed_purchase_per_group_df['Average Purchase Price'] = renamed_purchase_per_group_df['Total Purchase Value'] / renamed_purchase_per_group_df['Purchase Count']
renamed_purchase_per_group_df['Normalized Total Purchase Value'] = renamed_purchase_per_group_df['Total Purchase Value'] / (age_df.groupby('Age Group'))['SN'].count()
print(renamed_purchase_per_group_df)

           Purchase Count  Total Purchase Value  Average Purchase Price  \
Age Group                                                                 
<10                    28                 83.46                2.980714   
10-14                  35                 96.95                2.770000   
15-19                 133                386.42                2.905414   
20-24                 336                978.77                2.913006   
25-29                 125                370.33                2.962640   
30-34                  64                197.25                3.082031   
35-39                  42                119.40                2.842857   
40-44                  16                 51.03                3.189375   
45-49                   1                  2.72                2.720000   
50-54                   0                   NaN                     NaN   
55-59                   0                   NaN                     NaN   
60-64                   0

In [17]:
# Top Spenders

top_spenders = pd.DataFrame(json_df.groupby('SN')['Price'].sum().sort_values(ascending=False).head())
top_spenders = top_spenders.rename(columns={"Price": "Total Purchase Value"})
top_spenders['Purchase Count'] = (json_df.groupby('SN'))['Item ID'].count()
top_spenders['Average Purchase Price'] = top_spenders['Total Purchase Value'] / top_spenders['Purchase Count']
top_spenders

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Undirrala66,17.06,5,3.412000
Saedue76,13.56,4,3.390000
Mindimnya67,12.74,4,3.185000
Haellysu29,12.73,3,4.243333
Eoda93,11.58,3,3.860000


In [18]:
# Top Popular Items

top_items = pd.DataFrame(json_df.groupby(['Item ID', 'Item Name', 'Price'])['Item ID'].count().sort_values(ascending=False).head())
top_items = top_items.rename(columns={"Item ID": "Purchase Count"})
top_items['Total Purchase Value'] = json_df.groupby(['Item ID', 'Item Name', 'Price'])['Price'].sum()
top_items

,,,Purchase Count,Total Purchase Value
Item ID,Item Name,Price,,
84,Arcane Gem,2.23,11,24.53
39,"Betrayal, Whisper of Grieving Widows",2.35,11,25.85
31,Trickster,2.07,9,18.63
34,Retribution Axe,4.14,9,37.26
175,Woeful Adamantite Claymore,1.24,9,11.16


In [19]:
# Most Profitable Items

top_profitable_items = pd.DataFrame(json_df.groupby(['Item ID', 'Item Name', 'Price'])['Price'].sum().sort_values(ascending=False).head())
top_profitable_items = top_profitable_items.rename(columns={"Item ID": "Total Purchase Value"})
top_profitable_items['Total Purchase Value'] = json_df.groupby(['Item ID', 'Item Name', 'Price'])['Item ID'].count()
top_profitable_items

,,,Price,Total Purchase Value
Item ID,Item Name,Price,,
34,Retribution Axe,4.14,37.26,9
115,Spectral Diamond Doomblade,4.25,29.75,7
32,Orenmir,4.95,29.70,6
103,Singed Scalpel,4.87,29.22,6
107,"Splitter, Foe Of Subtlety",3.61,28.88,8
